In [1]:
"""
Name: movement.ipynb
Authors: Stephan Meighen-Berger, Martina Karl
Example how to create a video of the simulation
Requires mpl_toolkits and imageio
"""

'\nName: movement.ipynb\nAuthors: Stephan Meighen-Berger, Martina Karl\nExample how to create a video of the simulation\nRequires mpl_toolkits and imageio\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import matplotlib.colors as colors
import imageio

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day as FD
from fourth_day import config

In [6]:
# Scenario
config['scenario']['population size'] = 100
config['scenario']['duration'] = 720
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 30
config['scenario']['injection']['y range'] = [7.5, 12.5]
# Organisms
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 1.
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 40.,
    'y_length': 20.
}
# Water
config['water']['model']['name'] = 'custom'
config['water']['model']['directory'] = "../data/current/cyclic/"
# config['water']['model']['norm'] = 0.5
config['water']['model']['time step'] = 60
config['advanced']['starting step'] = 575
# config['advanced']['water grid size'] = 5e-2

In [7]:
# Creating a fourth_day object
fd = FD()

In [8]:
# Launching solver
fd.sim()

In [9]:
# Plotting standards
std_size = 15
fontsize = 20.
lw=3.
h_length=1.5

In [10]:
# Animation stack
fontsize = 12.
# The frames are created in a loop
cmap = plt.cm.inferno
norm = colors.Normalize(vmin=0., vmax=1.)
for i, _ in tqdm(enumerate(fd.t)):
    figure = plt.figure(figsize=(std_size, std_size * 6. / 8.))
    ax = figure.add_subplot(111)
    ax.scatter(fd.statistics[i].loc[:, 'pos_x'].values,
               fd.statistics[i].loc[:, 'pos_y'].values,
               color=cmap(norm(fd.statistics[i].loc[:, 'energy'].values)),
               s=30.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw, color='k')
    # Additional options
    ax.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax.title=ax.set_title('Population t = %.f s' %fd.t[i], fontsize=fontsize)
    ax.set_xlim(0., 40.)
    ax.set_ylim(0., 20.)
    figure.savefig(PICS + '\\Frames\\frame_%d.jpeg' %i, quality=50)
    plt.close(figure)

1440it [07:21,  2.10it/s]


In [13]:
images = []
filenames = np.array([
    PICS + 'Frames\\frame_%d.jpeg' %i
    for i in range(len(fd.t))
])
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(PICS + '%s.gif' %config['water']['model']['name'],
                images, fps=20)

KeyboardInterrupt: 